In [2]:
"""
Created on Thu Sep 08 11:15 2022

This script is to prepare the normalising coefficients and input data for the training on the whole dataset

Author: Clara Burgard
"""

'\nCreated on Thu Sep 08 11:15 2022\n\nThis script is to prepare the normalising coefficients and input data for the training on the whole dataset\n\nAuthor: Clara Burgard\n'

In [14]:
import numpy as np
import xarray as xr
from tqdm.notebook import trange, tqdm
import glob
import matplotlib as mpl
#import seaborn as sns
import datetime
import time

from nn_functions.constants import *
import nn_functions.prep_input_data as indat
from nn_functions.prep_input_data import compute_norm_metrics

import distributed

In [4]:
client = distributed.Client(n_workers=4, dashboard_address=':8795', local_directory='/tmp', memory_limit='4GB')

PREPARE THE CONTEXT OF THE INPUT DATA

In [5]:
inputpath_data = '/bettik/burgardc/DATA/NN_PARAM/interim/INPUT_DATA/' 
inputpath_data = '../../data/interim/INPUT_DATA/'

tblock_dim = range(1,14)
isf_dim = [10,11,12,13,18,22,23,24,25,30,31,33,38,39,40,42,43,44,45,47,48,51,52,53,54,55,58,61,65,66,69,70,71,73,75]
    

prepare metrics

In [18]:
from tqdm.notebook import trange, tqdm
import pandas as pd
import xarray as xr
import numpy as np

# Simple input data preparation
# Not bothering with cross validation
# Set data filepath
TS_opt = 'extrap'
inputpath_prof = inputpath_data+'EXTRAPOLATED_ISFDRAFT_CHUNKS/'
# Load in data
clean_df = pd.read_csv(inputpath_prof + 'dataframe_input_isf.csv')
# Clean data and transform to xarray
clean_df.reset_index(drop=True, inplace=True)
clean_ds = clean_df.to_xarray()
# NOt sure why but copy data set 
train_input_df = clean_ds.copy()
# More skipping cross validation
tblock_list = list(tblock_dim)
isf_list = list(isf_dim)
isf_val = isf_list
tt_val = tblock_list
# Copy dataset again to be the validation data
val_input_df = clean_ds.copy()
print('Loaded datasets')

## prepare input and target
y_train = train_input_df['melt_m_ice_per_y']
x_train = train_input_df.drop_vars(['melt_m_ice_per_y'])
y_val = val_input_df['melt_m_ice_per_y']
x_val = val_input_df.drop_vars(['melt_m_ice_per_y'])
print('x_train, y_train, x_val and y_val prepared')

# Normalise the input and output data
norm_summary_list = []
for norm_method in ['std','interquart','minmax']:
    summary_ds = compute_norm_metrics(x_train, y_train, norm_method)
    norm_summary_list.append(summary_ds)
summary_ds_all = xr.concat(norm_summary_list, dim='norm_method')
print('Data normalised for all three methods')

# Calculate var mean, var 
var_mean = summary_ds_all.sel(metric='mean_vars')
var_range = summary_ds_all.sel(metric='range_vars')
var_train_norm = (train_input_df - var_mean)/var_range
var_val_norm = (val_input_df - var_mean)/var_range
print('Normalisation metrics calculcated')

# Set data to variables and save to file
metrics_ds, var_train_norm, var_val_norm = summary_ds_all, var_train_norm, var_val_norm
outputpath_CVinput = inputpath_prof
metrics_ds.to_netcdf(outputpath_CVinput + 'metrics_norm_wholedataset.nc')
var_train_norm.to_netcdf(outputpath_CVinput + 'train_data_wholedataset.nc')
var_val_norm.to_netcdf(outputpath_CVinput + 'val_data_wholedataset.nc')    
print('Data saved')

Loaded datasets
x_train, y_train, x_val and y_val prepared
Data normalised for all three methods
Normalisation metrics calculcated
Data saved


In [11]:
# Run this block if you have created chunks for cross validation

tblock_dim = range(1,14)
isf_dim = [10,11,12,13,18,22,23,24,25,30,31,33,38,39,40,42,43,44,45,47,48,51,52,53,54,55,58,61,65,66,69,70,71,73,75]
TS_opt = 'extrap'

if TS_opt == 'extrap':
    outputpath_CVinput = inputpath_data+'EXTRAPOLATED_ISFDRAFT_CHUNKS/'
elif TS_opt == 'whole':
    outputpath_CVinput = inputpath_data+'WHOLE_PROF_CHUNKS/'
elif TS_opt == 'thermocline':
    outputpath_CVinput = inputpath_data+'THERMOCLINE_CHUNKS/'

print(outputpath_CVinput)

tblock_out = 0
isf_out = 0
metrics_ds, var_train_norm, var_val_norm = indat.prepare_input_data_CV(tblock_dim, isf_dim, tblock_out, isf_out, TS_opt, inputpath_data)
#metrics_ds = indat.prepare_input_data_CV_onlymetrics(tblock_dim, isf_dim, tblock_out, isf_out, TS_opt, inputpath_data)
metrics_ds.to_netcdf(outputpath_CVinput + 'metrics_norm_wholedataset.nc')
var_train_norm.to_netcdf(outputpath_CVinput + 'train_data_wholedataset.nc')
var_val_norm.to_netcdf(outputpath_CVinput + 'val_data_wholedataset.nc')    

../../data/interim/INPUT_DATA/EXTRAPOLATED_ISFDRAFT_CHUNKS/
1


FileNotFoundError: [Errno 2] No such file or directory: '../../data/interim/INPUT_DATA/EXTRAPOLATED_ISFDRAFT_CHUNKS/dataframe_input_isf010_001.csv'